In [1]:
#Once all the processed BLAST output TSVs are ready, merge all the files into one
import pandas as pd
import os

In [2]:
#Choose the directory containing all the sample tsv files
basedir = '/mnt/c/MinION_16S_barcodes/BLASTn/BLASTn/processed_tsv/'

In [14]:
#Read as list
read_tsv = [os.path.join(basedir, x) for x in os.listdir(basedir) if x.endswith('.tsv')]

In [15]:
len(read_tsv)

21

In [16]:
mergedf = pd.read_csv(read_tsv[0], sep='\t')

In [20]:
#Create a loop to merge all 21 files at once
for file in read_tsv[1:]:
    tmp_df = pd.read_csv(file, sep='\t')
    mergedf = pd.merge(mergedf,tmp_df, how = 'outer', left_on = 'primaryAccession', right_on = 'primaryAccession')
mergedf.fillna(value=0, inplace=True)

In [22]:
#Rename the sample column 
mergedf.rename(columns={'MF-143':'rabbit-143','MF-144': 'rabbit-144','MF-145':'rabbit-145','MF-154': 'rabbit-154','MF-157': 'hare-157','MF-138':'rabbit-138','MF-139': 'rabbit-139','MF-140':'rabbit-140','MF-141': 'rabbit-141','MF-142':'rabbit-142','MF-149': 'hare-149','MF-150':'hare-150','MF-151': 'hare-151','MF-152':'hare-152','MF-155': 'hare-155','MF-156':'hare-156','MF-136': 'hare-136','MF-148':'hare-148'}, inplace=True)

In [24]:
#Rename the column with primary accession numbers
mergedf.rename(columns={'primaryAccession': '#OTU ID'}, inplace=True)

In [26]:
#Total number of unique sequence IDs
mergedf['#OTU ID'].nunique()

75774

In [27]:
#Save the file in txt format
mergedf.to_csv(os.path.join(basedir, '20190823_merged_minion_feature_table.txt'),sep='\t',header=True,index =False)

In [ ]:
#Import the feature table into qiime2 and filter based on frequency (remove all sequences with frequency less than 2)
#Export the table and open it in jupyter notebook
#Process the taxonomy file to include only the accession numbers found in the feature table

In [50]:
basedir1 = '/mnt/c/MinION_16S_barcodes/QIIME_2/20190823_new/'

In [53]:
#join path to the frequency filtered feature table
read1_csv = os.path.join(basedir1, '20190823_downloadedfeaturetablefreq2.txt')
taxa1_df = pd.read_csv(read1_csv, sep= '\t')

In [55]:
#Download the silva_132 taxonomy file from SILVA website and join path to the file
basedir2 = '/mnt/c/MinION_16S_barcodes/BLASTn/BLASTn/taxonomy_files/'
read2_csv = os.path.join(basedir2, 'taxmap_slv_ssu_ref_132-corrected.txt')
taxa2_df = pd.read_csv(read2_csv, sep= '\t')

In [57]:
#Drop columns that are not required in the taxonomy map and eliminate duplicate Accession numbers
taxa2_df.drop(columns=['start','stop','organismName','taxid'], inplace=True)
taxa3_df = taxa2_df.drop_duplicates(subset='primaryAccession', keep='first', inplace=False)

In [59]:
#Total unique accession numbers found in the silva_132 taxonomy mapping file
taxa3_df['primaryAccession'].nunique()

2040731

In [60]:
#Merge the freaquency filtered dataframe with that of the taxonomy map file
merged_df_2 = pd.merge(taxa1_df,taxa3_df, how = 'left', left_on = '#OTU ID', right_on = 'primaryAccession')
merged_df_2.fillna(value='unassigned', inplace=True)

In [63]:
#drop all the sample columns. Only the column with feature ID and taxonomy lineage is required
merged_df_2.drop(columns=['primaryAccession', 'hare-136', 'rabbit-138', 'rabbit-139','rabbit-140','rabbit-141','rabbit-142','rabbit-143','rabbit-144','rabbit-145','rabbit-146','rabbit-147','hare-148','hare-149','hare-150','hare-151','hare-152','rabbit-153','rabbit-154','hare-155','hare-156','hare-157'],inplace=True)

In [65]:
#rename the column names to QIIME2 compatible form
merged_df_2.rename(columns={'path': 'Taxon'}, inplace=True)
merged_df_2.rename(columns={'#OTU ID': 'Feature ID'}, inplace=True)

In [66]:
#Save the file as TSV
merged_df_2.to_csv(os.path.join(basedir2, '20190823_taxamap_minION_blast.tsv'),sep='\t',header=True,index =False)